# Gemini Chatbot with Memory (Colab Example)

This project demonstrates how to build an **interactive chatbot with memory** using the **Google Gemini API** (`google-generativeai`).  
The chatbot keeps track of the conversation state, allowing for contextual and continuous dialogue.

---

## Features
- Uses **Gemini 2.0 Flash** model for fast responses.  
- Maintains **chat history** for contextual answers.  
- Simple interactive loop in Colab/terminal.  
- Shows memory state after each turn for debugging.

---

## Installation
Install the required dependency:

```bash
pip install google-generativeai


In [ ]:
# ==============================
# Step 1: Install dependencies
# ==============================
!pip install google-generativeai --quiet



In [ ]:
# ==============================
# Step 2: Import Libraries
# ==============================
import google.generativeai as genai
import textwrap


In [ ]:
# ==============================
# Step 3: Configure API Key directly
# ==============================
# API_KEY = ""  # your key here
# genai.configure(api_key=API_KEY)
#           (or)
# ==============================
# Step 3: Configure API Key thru Secrets
# ==============================
from google.colab import userdata   # for fetching secret key

# Retrieve API key stored in Colab's secrets
API_KEY = userdata.get('GEMINI_API_KEY')   # make sure you saved it with this name
genai.configure(api_key=API_KEY)


In [ ]:
# ==============================
# Step 4: Create Chatbot with Memory
# ==============================
# Chat session will hold memory (state)
model = genai.GenerativeModel("gemini-2.0-flash")
chat = model.start_chat(history=[])


In [ ]:
# ==============================
# Step 5: Helper to Display Messages
# ==============================
def pretty_print(text):
    print("\n".join(textwrap.wrap(text, width=100)))


In [ ]:
# ==============================
# Step 6: Start Interactive Chat
# ==============================
print(" Gemini Chatbot with Memory & State")
print("Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye! ")
        break

    response = chat.send_message(user_input)
    bot_reply = response.text

    print("Bot:", end=" ")
    pretty_print(bot_reply)

    # Debug: Show memory state
    print("\n[Memory State So Far]")
    for turn in chat.history:
      role = turn.role
    # parts is a list of Part objects, so we join their text
      parts_text = " ".join([p.text for p in turn.parts if hasattr(p, "text")])
    print(f"{role}: {parts_text}")
    print("-" * 60)



 Gemini Chatbot with Memory & State
Type 'exit' to stop.

You: my name is saravanan
Bot: Nice to meet you, Saravanan!

[Memory State So Far]
model: Nice to meet you, Saravanan!

------------------------------------------------------------
You: tell about you
Bot: I am a large language model, trained by Google.  I don't have a body, a personal history, or
emotions.  My purpose is to process information and respond to a wide range of prompts and
questions.  I can provide information on many topics, translate languages, write different kinds of
creative content, and answer your questions in an informative way, even if they are open ended,
challenging, or strange.  Think of me as a very advanced search engine combined with a powerful
writing tool. I learn from the massive dataset I was trained on, which includes a huge amount of
text and code.  I don't have experiences like humans do; my knowledge comes from the data I've
processed.  I'm always learning and improving, but I'm not perfect. 

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 657.98ms


Bot: Your name is Saravanan.

[Memory State So Far]
model: Your name is Saravanan.

------------------------------------------------------------


#  Gemini Chain-of-Thought Reasoning Agent (Colab)

This project demonstrates how to build an **interactive reasoning assistant** powered by **Google Gemini 2.0 Flash**.  
The agent is designed to **show its step-by-step reasoning** before giving a final concise answer.

---

##  Features
- Uses **Gemini 1.5 Flash** (fast + efficient).  
- Implements **Chain-of-Thought prompting** for transparent reasoning.  
- Includes **retry logic** (via `tenacity`) to handle API rate limits.  
- Reads **API key from Colab secrets** (safer than hardcoding).  
- Interactive loop for asking multiple questions.

---

##  Installation
Install required dependencies inside Colab:

```bash
pip install -q google-generativeai tenacity


In [ ]:
# ==============================
# Step 1: Install Gemini SDK
# ==============================
!pip install -q google-generativeai tenacity



In [ ]:
# ==============================
# Step 2: Imports
# ==============================
import os
import textwrap
import google.generativeai as genai
from tenacity import retry, wait_random_exponential, stop_after_attempt


In [ ]:
# ==============================
# Step 3: Load Gemini API Key from Colab secrets
# ==============================
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")  # must be set in Colab secrets
except ImportError:
    raise ImportError("This code requires Google Colab with secrets set.")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in Colab secrets. Please set it first.")

genai.configure(api_key=GEMINI_API_KEY)



In [ ]:
# ==============================
# Step 4: Initialize Gemini model
# ==============================
model = genai.GenerativeModel("gemini-2.0-flash")

# Retry logic for rate limits
@retry(wait=wait_random_exponential(multiplier=1, max=60), stop=stop_after_attempt(5))
def ask_gemini(prompt: str):
    response = model.generate_content(prompt)
    return response.text

def cot_agent_chain_of_thought(user_query: str) -> str:
    # Enhanced CoT prompt
    prompt = (
        "You are an expert reasoning assistant.\n"
        "When solving problems, follow these steps:\n"
        "1. Restate the question in your own words.\n"
        "2. Break the problem into smaller sub-steps.\n"
        "3. Think step by step, showing all reasoning explicitly.\n"
        "4. If math is needed, show calculations.\n"
        "5. If logic or assumptions are used, state them clearly.\n"
        "6. At the end, provide a concise final answer, starting with 'Answer:'.\n\n"
        f"Problem: {user_query}\n\n"
        "Begin reasoning:\n"
    )
    return ask_gemini(prompt)



In [ ]:
# ==============================
# Step 5: Run interactive loop
# ==============================
if __name__ == "__main__":
    print(" Gemini CoT Reasoning Agent")
    while True:
        query = input("\nEnter your question (or 'exit' to quit): ")
        if query.strip().lower() == "exit":
            print("Exiting. Happy reasoning! ")
            break
        try:
            output = cot_agent_chain_of_thought(query)
            print("\n---\n" + output + "\n---")
        except Exception as e:
            print(f"Failed to get response: {e}")
            print("Try again later or check quota/billing.")






